In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import sys

In [2]:
inputString = [2,45,30,55,10]
outString = [25,30,55,10,1]

In [3]:
numFeatures = 100
vocabSize = 80

In [4]:
#define embedding for each input vector of size (100,1) random numbers
embedding = []
for i  in range(len(inputString)):
    a = np.random.rand(numFeatures,1)
    embedding.append(a)
     

In [5]:
len(embedding)

5

In [6]:
# define the one hot encoding of vocab size
def OneHotEncoding(idx):
    var = np.zeros((vocabSize,1),dtype=float)
    var[idx] = 1
    return var

In [7]:
numUnits = 50
h0 = torch.tensor(np.zeros((numUnits,1)))
Wh = torch.tensor(np.random.uniform(0,1,(numUnits,numUnits)),requires_grad = True)
Wx = torch.tensor(np.random.uniform(0,1,(numUnits,numFeatures)),requires_grad = True)
Wy = torch.tensor(np.random.uniform(0,1,(vocabSize,numUnits)),requires_grad = True)

In [8]:
#define stepforward function input:  xt,wx,wh,wy,prevMemory ; output : ht,y_hat
def stepForward(xt,Wx,Wh,Wy,prevMemory):
    x_frd = torch.matmul(Wx,torch.from_numpy(xt))
    h_frd = torch.matmul(Wh,prevMemory)
    ht = torch.tanh(x_frd + h_frd)
    y_hat = F.softmax(torch.matmul(Wy,ht),dim=0)
    return ht,y_hat

In [9]:
# calculate the y_hat and ht with step_forward
ht,y_hat = stepForward(embedding[0],Wx,Wh,Wy,h0)

In [10]:
ht.shape

torch.Size([50, 1])

In [11]:
y_hat.shape

torch.Size([80, 1])

In [12]:
# define fullforwardpass function ; input :- X,Wx,Wh,Wy,previousMemory ; output: y_hat matrix
def fullForward(X,Wx,Wh,Wy,prevMemory):
    y_hat = []
    for i in range(len(X)):
        ht,yhat = stepForward(X[i],Wx,Wh,Wy,prevMemory)
        prevMemory = ht
        y_hat.append(yhat)
    return y_hat

In [13]:
y_hat = fullForward(embedding,Wx,Wh,Wy,h0)

In [14]:
y_hat[0].shape

torch.Size([80, 1])

In [15]:
# define a computeLoss function that computes the log loss ; input: y,y_hat and output : loss
def computeLoss(y,y_hat):
    loss = 0
    for yi,yi_hat in zip(y,y_hat):
        lt = -torch.log2(yi_hat[yi==1])
        loss += lt
    return loss/len(y)

In [16]:
# get y to compute the loss ; hint : it would be oneHot vector of output string index
y = []
for idx in outString:
    y.append(OneHotEncoding(idx))

In [17]:
print(computeLoss(y,y_hat))

tensor([9.9471], dtype=torch.float64, grad_fn=<DivBackward0>)


In [40]:
def updateParams(Wx,Wh,Wy,dWx,dWh,dWy,lr):
    with torch.no_grad():
        Wx -= lr*dWx
        Wy -= lr*dWy
        Wh -= lr*dWh
    return Wx,Wh,Wy

In [41]:
# define the trainRNN function ; input : X,y,Wx,Wh,Wy,prevMemory ; output : losses and weights
def trainRNN(X,y,Wx,Wh,Wy,prevMemory,lr,nepochs):
    losses = []
    for epoch in range(nepochs):
        y_hat = fullForward(X,Wx,Wh,Wy,prevMemory)
        loss = computeLoss(y,y_hat)
        loss.backward()
        losses.append(loss)
        print("Loss after epoch %d : %f" %(epoch,loss))
        sys.stdout.flush()
        dWx = Wx.grad.data
        dWh = Wh.grad.data
        dWy  = Wy.grad.data
        Wx,Wh,Wy = updateParams(Wx,Wh,Wy,dWx,dWh,dWy,lr)
        Wx.grad.data.zero_()
        Wh.grad.data.zero_()
        Wy.grad.data.zero_()
    return Wx,Wh,Wy, losses

In [42]:
Wx,Wh,Wy,losses = trainRNN(embedding,y,Wx,Wh,Wy,h0,0.001,100)

Loss after epoch 0 : 7.355248
Loss after epoch 1 : 7.311033
Loss after epoch 2 : 7.289034
Loss after epoch 3 : 7.267089
Loss after epoch 4 : 7.245198
Loss after epoch 5 : 7.223363
Loss after epoch 6 : 7.201583
Loss after epoch 7 : 7.179859
Loss after epoch 8 : 7.158192
Loss after epoch 9 : 7.136580
Loss after epoch 10 : 7.115026
Loss after epoch 11 : 7.093528
Loss after epoch 12 : 7.072089
Loss after epoch 13 : 7.050707
Loss after epoch 14 : 7.029383
Loss after epoch 15 : 7.008117
Loss after epoch 16 : 6.986911
Loss after epoch 17 : 6.965764
Loss after epoch 18 : 6.944676
Loss after epoch 19 : 6.923648
Loss after epoch 20 : 6.902681
Loss after epoch 21 : 6.881774
Loss after epoch 22 : 6.860928
Loss after epoch 23 : 6.840143
Loss after epoch 24 : 6.819419
Loss after epoch 25 : 6.798758
Loss after epoch 26 : 6.778158
Loss after epoch 27 : 6.757621
Loss after epoch 28 : 6.737147
Loss after epoch 29 : 6.716736
Loss after epoch 30 : 6.696389
Loss after epoch 31 : 6.676105
Loss after epoch 3